In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

### Recreate the Vine DF

In [4]:
from pyspark.sql.functions import to_date

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine", "verified_purchase"])

vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

### Perform bias analysis

In [7]:
vine_df_filtered = vine_df.filter((vine_df["total_votes"] > 20) & (vine_df["helpful_votes"]/vine_df["total_votes"] > 0.5))
vine_df_filtered.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|     

In [9]:
vine_df_paid = vine_df_filtered.filter(vine_df_filtered["verified_purchase"] == "Y")
vine_df_paid.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|                Y|
|R16621F16PQN5A|          3|           24|         31|   N|                Y|
| RNEMGSUKJ4535|          2|           21|         25|   N|                Y|
|R3R2UKBFKP1IQ4|          1|           19|         23|   N|                Y|
|R2T90XM5G2DBZA|          3|           15|         22|   N|                Y|
|R12E2W0XOV3I2S|          1|           24|         30|   N|     

In [10]:
vine_df_unpaid = vine_df_filtered.filter(vine_df_filtered["verified_purchase"] == "N")
vine_df_unpaid.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R2OWDAR0EDEPTF|          1|           23|         28|   N|                N|
|R29BOS5HMAY1LO|          5|           88|        110|   N|                N|
| R8WD0136CETA7|          2|           99|        119|   N|                N|
|R1LXBPG1M9DDOC|          1|          102|        106|   N|                N|
| R5ZMK0Y0KRBRZ|          1|           79|         91|   N|     

In [11]:
# Determine the total number of reviews
vine_df_filtered.count()

37475

In [20]:
# Number of 5-star reviews

## Total
print("Total 5-star reviews = " + str(vine_df_filtered.filter(vine_df_filtered['star_rating'] == 5).count()))

## Paid
print("Total Paid 5-star reviews = " + str(vine_df_paid.filter(vine_df_paid['star_rating'] == 5).count()))

## Unpaid
print("Total Unpaid 5-star reviews = "+ str(vine_df_unpaid.filter(vine_df_unpaid['star_rating'] == 5).count()))

Total 5-star reviews = 14670
Total Paid 5-star reviews = 3594
Total Unpaid 5-star reviews = 11076


In [22]:
# Percentage of 5-star reviews for the two types of review (paid vs unpaid).

print("Percentage of 5-star reviews on verified purchases = "+ str(((vine_df_paid.filter(vine_df_paid['star_rating'] == 5).count()) / (vine_df_paid.count())*100))) 

print("Percentage of 5-star reviews on not verified purchases = "+ str(((vine_df_unpaid.filter(vine_df_unpaid['star_rating'] == 5).count()) / (vine_df_unpaid.count())*100))) 


Percentage of 5-star reviews on verified purchases = 38.41795831106361
Percentage of 5-star reviews on not verified purchases = 39.388335704125176
